In [ ]:
! pip install git+https://github.com/tagucci/pythonrouge.git
! pip install janome

  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-p2r706n8
  Running command git clone --filter=blob:none --quiet https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-p2r706n8
  Resolved https://github.com/tagucci/pythonrouge.git to commit 0f1603dbc089ecb0fb40fdd3a28576e9f3d36769
  Preparing metadata (setup.py) ... done


In [ ]:
!apt-get install -y cpanminus
!cpanm --force XML::Parser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cpanminus is already the newest version (1.7045-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--> Working on XML::Parser
Fetching http://www.cpan.org/authors/id/T/TO/TODDR/XML-Parser-2.46.tar.gz ... OK
Configuring XML-Parser-2.46 ... OK
Building and testing XML-Parser-2.46 ... OK
Successfully reinstalled XML-Parser-2.46
1 distribution installed


In [ ]:
from __future__ import print_function
from collections import defaultdict
from pythonrouge.pythonrouge import Pythonrouge
from pprint import pprint
from janome.tokenizer import Tokenizer

def word2ids(summary, reference):
    id_dict = defaultdict(lambda: len(id_dict))
    summary = [[[' '.join([str(id_dict[w]) for w in sent.split()])
                   for sent in doc] for doc in refs] for refs in summary]
    reference = [[[' '.join([str(id_dict[w]) for w in sent.split()])
                   for sent in doc] for doc in refs] for refs in reference]
    return summary, reference

def tokenize_sentences(sentences):
    tokenizer = Tokenizer()
    return [' '.join(token.surface for token in tokenizer.tokenize(sentence)) for sentence in sentences]

if __name__ == '__main__':
    # ファイルからデータを読み込んでjanomeを使用して単語分割し、要約と参照要約に格納する
    with open('/content/generated.txt', mode='r', encoding='utf-8') as f:
        summary_lines = f.readlines()
        summary = []
        for line in summary_lines:
            summary.append([tokenize_sentences(sent.split(' ')) for sent in line.split('。') if sent.strip()])

    with open('/content/test.txt', mode='r', encoding='utf-8') as f:
        reference_lines = f.readlines()
        reference = []
        for line in reference_lines:
            reference.append([tokenize_sentences(sent.split(' ')) for sent in line.split('。') if sent.strip()])

    summary, reference = word2ids(summary, reference)
    rouge = Pythonrouge(summary_file_exist=False,
                        summary=summary, reference=reference,
                        n_gram=2, ROUGE_SU4=False, ROUGE_L=True,
                        recall_only=True)
    score = rouge.calc_score()

    # 結果の出力
    pprint(score)

{'ROUGE-1': 0.61334, 'ROUGE-2': 0.30815, 'ROUGE-L': 0.51927}
